In [1]:
import requests as r

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
nativead = pd.read_excel("../data/Nativead.xlsx")
nativead.head()

,Site,Status,Coder,Story 1,Story 2,Story 3
0,Yahoo News,DONE,DI,https://goingthere.yahoo.com/post/133803688011...,https://goingthere.yahoo.com/post/130087715036...,https://goingthere.yahoo.com/post/130160620511...
1,ABC News \n,Done,DI,http://abcnews.go.com/US/wireStory/providence-...,http://abcnews.go.com/Politics/obama-hits-trum...,http://abcnews.go.com/Politics/justice-antonin...
2,CNN Network,DONE,BWW,http://sponsorcontent.cnn.com/interactive/cine...,http://sponsorcontent.cnn.com/amazon/article/8...,http://sponsorcontent.cnn.com/subaru/article/a...
3,NBC News Digital,DONE,NJE,http://www.nbcnews.com/video/this-designer-cut...,http://www.nbcnews.com/video/baby-boomers-are-...,http://www.nbcnews.com/video/he-creates-furnit...
4,HuffingtonPost.com,DONE,BWW,http://www.huffingtonpost.com/2015/06/04/best-...,http://www.huffingtonpost.com/kathy-magee/a-sm...,http://www.huffingtonpost.com/2015/05/29/evolu...


In [3]:
nativead['Status'].value_counts()

DONE                38
Done                 5
N/A, now defunct     1
Found only 1         1
Name: Status, dtype: int64

In [4]:
nativead['Status'] = nativead['Status'].astype(str)
nativead = nativead[nativead['Status'].apply(str.lower)=='done']
nativead['Status'].value_counts()

DONE    38
Done     5
Name: Status, dtype: int64

In [5]:
nativead = nativead.drop(['Status', 'Coder'], axis=1)
nativead.head()

,Site,Story 1,Story 2,Story 3
0,Yahoo News,https://goingthere.yahoo.com/post/133803688011...,https://goingthere.yahoo.com/post/130087715036...,https://goingthere.yahoo.com/post/130160620511...
1,ABC News \n,http://abcnews.go.com/US/wireStory/providence-...,http://abcnews.go.com/Politics/obama-hits-trum...,http://abcnews.go.com/Politics/justice-antonin...
2,CNN Network,http://sponsorcontent.cnn.com/interactive/cine...,http://sponsorcontent.cnn.com/amazon/article/8...,http://sponsorcontent.cnn.com/subaru/article/a...
3,NBC News Digital,http://www.nbcnews.com/video/this-designer-cut...,http://www.nbcnews.com/video/baby-boomers-are-...,http://www.nbcnews.com/video/he-creates-furnit...
4,HuffingtonPost.com,http://www.huffingtonpost.com/2015/06/04/best-...,http://www.huffingtonpost.com/kathy-magee/a-sm...,http://www.huffingtonpost.com/2015/05/29/evolu...


In [6]:
def get_article_text(urls=pd.DataFrame):
    return np.random.choice(urls.to_numpy().flatten())

story_urls = nativead[['Story 1', 'Story 2', 'Story 3']]
article = r.get(get_article_text(story_urls))
while not article:
    print('Response code 404, trying again...')
    article = r.get(get_article_text(story_urls))
else:
    data = article.text
    soup = BeautifulSoup(data)

Response code 404, trying again...


In [7]:
np.random.choice(soup.find_all('p'))

<p>The way he keeps it all straight is by theming his days, according to <em><a class="" href="http://www.forbes.com/sites/kevinkruse/2015/10/12/jack-dorsey-productivity-secret/" rel="noopener noreferrer" target="_blank">Forbes</a></em>: Monday is for management and running the companies (both companies); Tuesday is all about the product; Wednesday is for marketing and growth; Thursday is for partnerships; Friday is company culture and recruiting; Saturday is time off, and Sunday is for reflection and strategy.</p>